In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f631e6f7910; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'category': 'category',
 'cust_id': 'id',
 'product_id': 'prod_id',
 'quantity': 'purchasequantity',
 'timestamp': 'date'}

In [5]:
query = 'SELECT "%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                               doc['column_map']['TRANSACTION_MASTER']['category'],
                               doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "id","category" FROM "trans"


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

In [9]:
col_names = [desc[0] for desc in temp.cur.description]

In [10]:
col_names

['id', 'category']

In [11]:
import pandas as pd

In [12]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [13]:
trans_table.head()

,id,category
0,2255653295,5824
1,2255653295,416
2,2255653295,2633
3,2255653295,2613
4,2255653295,3611


In [14]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['category']:'category'}, inplace=True)

In [15]:
trans_table.head()

,cust_id,category
0,2255653295,5824
1,2255653295,416
2,2255653295,2633
3,2255653295,2613
4,2255653295,3611


In [16]:
trans_table.shape

(71932603, 2)

In [17]:
#trans_table = trans_table[['cust_id','category']]

In [18]:
len(trans_table['cust_id'].unique())

62308

In [19]:
grp_cust = trans_table.groupby('cust_id').count().reset_index()

In [20]:
grp_cust.head()

,cust_id,category
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [21]:
grp_cust.shape

(62308, 2)

In [22]:
grp_cust.rename(columns={'category':'counter'}, inplace=True)

In [23]:
grp_cust.head()

,cust_id,counter
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [24]:
grp_cust_df_1 = grp_cust.query('counter != 1')

In [25]:
grp_cust_df_1.shape

(62301, 2)

In [26]:
grp_cust.shape

(62308, 2)

In [27]:
grp_cust_df_1.reset_index(drop=True, inplace=True)

In [28]:
grp_cust_df_1.head()

,cust_id,counter
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [29]:
grp_cust_df_1.tail()

,cust_id,counter
62296,99804794,529
62297,99818613,673
62298,99844671,1656
62299,99926227,914
62300,99972977,670


In [30]:
cust = grp_cust_df_1[['cust_id']]

In [31]:
cust.head()

,cust_id
0,100010021
1,100022923
2,100029473
3,100050424
4,100051316


In [32]:
final_data = trans_table.merge(cust, on='cust_id')

In [33]:
final_data.shape

(71932596, 2)

In [34]:
cust_categ_comb = final_data.groupby(['cust_id','category']).size().reset_index().rename(columns={0:'counter'})

In [35]:
cust_categ_comb.head()

,cust_id,category,counter
0,100010021,201,1
1,100010021,311,1
2,100010021,410,1
3,100010021,416,8
4,100010021,417,1


In [36]:
affinity_df = cust_categ_comb.merge(grp_cust_df_1, on='cust_id')

In [37]:
affinity_df.head()

,cust_id,category,counter_x,counter_y
0,100010021,201,1,349
1,100010021,311,1,349
2,100010021,410,1,349
3,100010021,416,8,349
4,100010021,417,1,349


In [38]:
affinity_df.rename(columns={'counter_x':'combination'}, inplace=True)
affinity_df.rename(columns={'counter_y':'visits'}, inplace=True)

In [39]:
affinity_df.head()

,cust_id,category,combination,visits
0,100010021,201,1,349
1,100010021,311,1,349
2,100010021,410,1,349
3,100010021,416,8,349
4,100010021,417,1,349


In [40]:
affinity_df['affinity'] = affinity_df['combination'] / affinity_df['visits'] * 100

In [41]:
affinity_df.head()

,cust_id,category,combination,visits,affinity
0,100010021,201,1,349,0.286533
1,100010021,311,1,349,0.286533
2,100010021,410,1,349,0.286533
3,100010021,416,8,349,2.292264
4,100010021,417,1,349,0.286533


In [42]:
affinity_df['affinity_round'] = affinity_df.affinity.round()

In [43]:
affinity_df.head()

,cust_id,category,combination,visits,affinity,affinity_round
0,100010021,201,1,349,0.286533,0.0
1,100010021,311,1,349,0.286533,0.0
2,100010021,410,1,349,0.286533,0.0
3,100010021,416,8,349,2.292264,2.0
4,100010021,417,1,349,0.286533,0.0


In [44]:
affinity_user_df = affinity_df[['cust_id','affinity_round']]

In [45]:
affinity_user_df.head()

,cust_id,affinity_round
0,100010021,0.0
1,100010021,0.0
2,100010021,0.0
3,100010021,2.0
4,100010021,0.0


In [46]:
affinity_user_df.shape

(11965910, 2)

In [47]:
affinity_user_df1 = affinity_user_df.drop_duplicates('cust_id').reset_index(drop = True)

In [48]:
affinity_user_df1.shape

(62301, 2)

In [49]:
affinity_user_comb = affinity_user_df1.groupby(['affinity_round','cust_id']).size().reset_index().rename(columns={0:'users_count'})

In [50]:
affinity_user_comb.head()

,affinity_round,cust_id,users_count
0,0.0,100010021,1
1,0.0,100022923,1
2,0.0,100029473,1
3,0.0,100050424,1
4,0.0,100051316,1


In [51]:
affinity_user_comb.tail()

,affinity_round,cust_id,users_count
62296,50.0,4053422289,1
62297,50.0,4377662924,1
62298,50.0,4396488884,1
62299,50.0,4670979967,1
62300,100.0,2595430565,1


In [52]:
affinity_user_comb_df = affinity_user_comb.groupby(by=['affinity_round'])['users_count'].sum()

In [53]:
affinity_user_comb_df.head()

affinity_round
0.0    24583
1.0    15709
2.0     5986
3.0     3706
4.0     2867
Name: users_count, dtype: int64

In [54]:
type(affinity_user_comb_df)

pandas.core.series.Series

In [55]:
affinity_user_comb_df = affinity_user_comb_df.to_frame()

In [56]:
affinity_user_comb_df.head()

,users_count
affinity_round,
0.0,24583
1.0,15709
2.0,5986
3.0,3706
4.0,2867


In [57]:
affinity_user_comb_df.reset_index(inplace=True)

In [58]:
affinity_user_comb_df

,affinity_round,users_count
0,0.0,24583
1,1.0,15709
2,2.0,5986
3,3.0,3706
4,4.0,2867
5,5.0,2285
6,6.0,1656
7,7.0,1310
8,8.0,886
9,9.0,660


In [59]:
affinity_user_comb_df.users_count.sum()

62301

In [60]:
affinity_user_comb_df.sort_index(ascending=False,inplace=True)

In [61]:
affinity_user_comb_df.head()

,affinity_round,users_count
40,100.0,1
39,50.0,5
38,48.0,1
37,45.0,2
36,38.0,1


In [62]:
affinity_user_comb_df['user_per'] = affinity_user_comb_df['users_count']/affinity_user_comb_df['users_count'].sum() * 100

In [63]:
affinity_user_comb_df.head()

,affinity_round,users_count,user_per
40,100.0,1,0.001605
39,50.0,5,0.008026
38,48.0,1,0.001605
37,45.0,2,0.003210
36,38.0,1,0.001605


In [64]:
affinity_user_comb_df['aggregated_per'] = affinity_user_comb_df.user_per.cumsum()

In [65]:
affinity_user_comb_df.head()

,affinity_round,users_count,user_per,aggregated_per
40,100.0,1,0.001605,0.001605
39,50.0,5,0.008026,0.009631
38,48.0,1,0.001605,0.011236
37,45.0,2,0.003210,0.014446
36,38.0,1,0.001605,0.016051


In [66]:
affinity_user_comb_df

,affinity_round,users_count,user_per,aggregated_per
40,100.0,1,0.001605,0.001605
39,50.0,5,0.008026,0.009631
38,48.0,1,0.001605,0.011236
37,45.0,2,0.003210,0.014446
36,38.0,1,0.001605,0.016051
35,37.0,1,0.001605,0.017656
34,35.0,1,0.001605,0.019261
33,33.0,3,0.004815,0.024077
32,32.0,1,0.001605,0.025682
31,31.0,2,0.003210,0.028892


In [67]:
affinity_user_comb_df['round_aggr'] = affinity_user_comb_df.aggregated_per.round()

In [68]:
affinity_user_comb_df.head()

,affinity_round,users_count,user_per,aggregated_per,round_aggr
40,100.0,1,0.001605,0.001605,0.0
39,50.0,5,0.008026,0.009631,0.0
38,48.0,1,0.001605,0.011236,0.0
37,45.0,2,0.003210,0.014446,0.0
36,38.0,1,0.001605,0.016051,0.0


In [69]:
affinity_user_comb_df.tail()

,affinity_round,users_count,user_per,aggregated_per,round_aggr
4,4.0,2867,4.601852,19.770148,20.0
3,3.0,3706,5.948540,25.718688,26.0
2,2.0,5986,9.608192,35.326881,35.0
1,1.0,15709,25.214684,60.541564,61.0
0,0.0,24583,39.458436,100.000000,100.0


In [70]:
affinity_user_comb_df

,affinity_round,users_count,user_per,aggregated_per,round_aggr
40,100.0,1,0.001605,0.001605,0.0
39,50.0,5,0.008026,0.009631,0.0
38,48.0,1,0.001605,0.011236,0.0
37,45.0,2,0.003210,0.014446,0.0
36,38.0,1,0.001605,0.016051,0.0
35,37.0,1,0.001605,0.017656,0.0
34,35.0,1,0.001605,0.019261,0.0
33,33.0,3,0.004815,0.024077,0.0
32,32.0,1,0.001605,0.025682,0.0
31,31.0,2,0.003210,0.028892,0.0


In [71]:
a = affinity_user_comb_df.loc[affinity_user_comb_df['round_aggr'] <= 2]

In [72]:
a

,affinity_round,users_count,user_per,aggregated_per,round_aggr
40,100.0,1,0.001605,0.001605,0.0
39,50.0,5,0.008026,0.009631,0.0
38,48.0,1,0.001605,0.011236,0.0
37,45.0,2,0.003210,0.014446,0.0
36,38.0,1,0.001605,0.016051,0.0
35,37.0,1,0.001605,0.017656,0.0
34,35.0,1,0.001605,0.019261,0.0
33,33.0,3,0.004815,0.024077,0.0
32,32.0,1,0.001605,0.025682,0.0
31,31.0,2,0.003210,0.028892,0.0


In [73]:
a.reset_index(drop=True, inplace=True)

In [74]:
a

,affinity_round,users_count,user_per,aggregated_per,round_aggr
0,100.0,1,0.001605,0.001605,0.0
1,50.0,5,0.008026,0.009631,0.0
2,48.0,1,0.001605,0.011236,0.0
3,45.0,2,0.003210,0.014446,0.0
4,38.0,1,0.001605,0.016051,0.0
5,37.0,1,0.001605,0.017656,0.0
6,35.0,1,0.001605,0.019261,0.0
7,33.0,3,0.004815,0.024077,0.0
8,32.0,1,0.001605,0.025682,0.0
9,31.0,2,0.003210,0.028892,0.0


In [75]:
a_value = a.affinity_round.loc[a['round_aggr'] <= 2]

In [76]:
a_value = a_value.tolist()

In [77]:
a_value

[100.0,
 50.0,
 48.0,
 45.0,
 38.0,
 37.0,
 35.0,
 33.0,
 32.0,
 31.0,
 30.0,
 29.0,
 28.0,
 27.0,
 26.0,
 25.0,
 24.0,
 23.0,
 22.0,
 21.0,
 20.0,
 19.0,
 18.0,
 17.0,
 16.0,
 15.0,
 14.0,
 13.0]

In [78]:
if len(a_value) == 0:
    min_a = None
else:
    min_a = min(a_value)

In [79]:
min_a

13.0

In [80]:
b = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >2) & 
                                       (affinity_user_comb_df['round_aggr'] <=10)]

In [81]:
b

,affinity_round,users_count,user_per,aggregated_per,round_aggr
12,12.0,398,0.638834,2.701401,3.0
11,11.0,418,0.670936,3.372338,3.0
10,10.0,552,0.886021,4.258359,4.0
9,9.0,660,1.059373,5.317732,5.0
8,8.0,886,1.422128,6.739860,7.0
7,7.0,1310,2.102695,8.842555,9.0


In [82]:
b.reset_index(drop=True, inplace=True)

In [83]:
b

,affinity_round,users_count,user_per,aggregated_per,round_aggr
0,12.0,398,0.638834,2.701401,3.0
1,11.0,418,0.670936,3.372338,3.0
2,10.0,552,0.886021,4.258359,4.0
3,9.0,660,1.059373,5.317732,5.0
4,8.0,886,1.422128,6.739860,7.0
5,7.0,1310,2.102695,8.842555,9.0


In [84]:
b_value = b.affinity_round.loc[(b['round_aggr'] >2) & 
                          (b['round_aggr'] <=10)]

In [85]:
b_value

0    12.0
1    11.0
2    10.0
3     9.0
4     8.0
5     7.0
Name: affinity_round, dtype: float64

In [86]:
b_value = b_value.tolist()

In [87]:
b_value

[12.0, 11.0, 10.0, 9.0, 8.0, 7.0]

In [88]:
if len(b_value) == 0:
    min_b = None
else:
    min_b = min(b_value)

In [89]:
min_b

7.0

In [90]:
c = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >10) & 
                                       (affinity_user_comb_df['round_aggr'] <=25)]

In [91]:
c

,affinity_round,users_count,user_per,aggregated_per,round_aggr
6,6.0,1656,2.658063,11.500618,12.0
5,5.0,2285,3.667678,15.168296,15.0
4,4.0,2867,4.601852,19.770148,20.0


In [92]:
c.reset_index(drop=True, inplace=True)

In [93]:
c_value = c.affinity_round.loc[(c['round_aggr'] >10) & 
                                       (c['round_aggr'] <=25)]

In [94]:
c_value

0    6.0
1    5.0
2    4.0
Name: affinity_round, dtype: float64

In [95]:
c_value = c_value.tolist()

In [96]:
c_value

[6.0, 5.0, 4.0]

In [97]:
if len(c_value) == 0:
    min_c = None
else:
    min_c = min(c_value)

In [98]:
min_c

4.0

In [99]:
d = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >25) & 
                                       (affinity_user_comb_df['round_aggr'] <=50)]

In [100]:
d

,affinity_round,users_count,user_per,aggregated_per,round_aggr
3,3.0,3706,5.948540,25.718688,26.0
2,2.0,5986,9.608192,35.326881,35.0


In [101]:
d.reset_index(drop=True, inplace=True)

In [102]:
d_value = d.affinity_round.loc[(d['round_aggr'] > 25) & 
                          (d['round_aggr'] <=50)]

In [103]:
d_value

0    3.0
1    2.0
Name: affinity_round, dtype: float64

In [104]:
d_value= d_value.tolist()

In [105]:
d_value

[3.0, 2.0]

In [106]:
if len(d_value) == 0:
    min_d = None
else:
    min_d = min(d_value)

In [107]:
min_d

2.0

In [108]:
e = affinity_user_comb_df.loc[affinity_user_comb_df['round_aggr'] > 50]

In [109]:
e.reset_index(drop=True, inplace=True)

In [110]:
e_value = e.affinity_round.loc[e['round_aggr'] > 50]

In [111]:
e_value

0    1.0
1    0.0
Name: affinity_round, dtype: float64

In [112]:
e_value = e_value.tolist()

In [113]:
e_value

[1.0, 0.0]

In [114]:
if len(e_value) == 0:
    min_e = None
else:
    min_e = min(e_value)

In [115]:
min_e

0.0

In [116]:
affinity_df['recommendation_class'] = ""

In [117]:
affinity_df.head()

,cust_id,category,combination,visits,affinity,affinity_round,Recommendation_class
0,100010021,201,1,349,0.286533,0.0,
1,100010021,311,1,349,0.286533,0.0,
2,100010021,410,1,349,0.286533,0.0,
3,100010021,416,8,349,2.292264,2.0,
4,100010021,417,1,349,0.286533,0.0,


In [118]:
if min_a is not None:
    affinity_df.loc[affinity_df['affinity_round'] >= min_a, 'recommendation_class'] = "Class_A"

In [119]:
affinity_df.head()

,cust_id,category,combination,visits,affinity,affinity_round,Recommendation_class
0,100010021,201,1,349,0.286533,0.0,
1,100010021,311,1,349,0.286533,0.0,
2,100010021,410,1,349,0.286533,0.0,
3,100010021,416,8,349,2.292264,2.0,
4,100010021,417,1,349,0.286533,0.0,


In [120]:
if min_b is not None and min_a is not None:
    affinity_df.loc[(affinity_df['affinity_round'] >= min_b) & (affinity_df['affinity_round'] < min_a), 'recommendation_class'] = "Class_B"

In [121]:
if min_c is not None and min_b is not None:
    affinity_df.loc[(affinity_df['affinity_round'] >= min_c) & (affinity_df['affinity_round'] < min_b), 'recommendation_class'] = "Class_C"

In [122]:
if min_d is not None and min_c is not None:
    affinity_df.loc[(affinity_df['affinity_round'] >= min_d) & (affinity_df['affinity_round'] < min_c), 'recommendation_class'] = "Class_D"

In [123]:
if min_e is not None:
    affinity_df.loc[(affinity_df['affinity_round'] >= min_e) & (affinity_df['affinity_round'] < min_d),'recommendation_class'] = "Class_E"

In [124]:
affinity_df.head()

,cust_id,category,combination,visits,affinity,affinity_round,Recommendation_class
0,100010021,201,1,349,0.286533,0.0,Class_E
1,100010021,311,1,349,0.286533,0.0,Class_E
2,100010021,410,1,349,0.286533,0.0,Class_E
3,100010021,416,8,349,2.292264,2.0,Class_D
4,100010021,417,1,349,0.286533,0.0,Class_E


In [125]:
affinity_df.shape

(11965910, 7)

In [126]:
len(affinity_df.cust_id.unique())

62301

In [127]:
del affinity_df['combination']

In [128]:
del affinity_df['visits']

In [129]:
del affinity_df['affinity']

In [130]:
del affinity_df['affinity_round']

In [131]:
affinity_df.head()

,cust_id,category,Recommendation_class
0,100010021,201,Class_E
1,100010021,311,Class_E
2,100010021,410,Class_E
3,100010021,416,Class_D
4,100010021,417,Class_E
